In [869]:
import numpy as np
import pandas as pd
import random

In [870]:
gbif = pd.read_csv("/home/joon/Téléchargements/0025255-190415153152247.csv",sep='	')

In [871]:
gbif.columns

Index(['taxonKey', 'scientificName', 'acceptedTaxonKey',
       'acceptedScientificName', 'numberOfOccurrences', 'taxonRank',
       'taxonomicStatus', 'kingdom', 'kingdomKey', 'phylum', 'phylumKey',
       'class', 'classKey', 'order', 'orderKey', 'family', 'familyKey',
       'genus', 'genusKey', 'species', 'speciesKey'],
      dtype='object')

In [872]:
gbif = gbif_raw[["species","scientificName","acceptedScientificName"]]

In [873]:
iucn = pd.read_csv("/home/joon/Téléchargements/iucn/taxonomy.csv",sep=',')

In [874]:
iucn.columns

Index(['kingdomName', 'phylumName', 'orderName', 'className', 'familyName',
       'genusName', 'speciesName', 'infraType', 'infraName', 'infraAuthority',
       'authority', 'internalTaxonId', 'taxonomicNotes'],
      dtype='object')

In [875]:
iucn = iucn[["genusName","speciesName","authority"]]

Il y a un encodage bizarre pour "&" dans iucn. On corrige ça:

In [876]:
iucn['authority'] = iucn['authority'].str.replace('&amp;',"&",regex=False)

On définit une fonction qui permet de filtrer les lignes d'une sous-dataFrame de `gbif` contenant `string` dans les colonnes `scientificName` ou `acceptedScientificName`.

In [877]:
def search_in_gbif_names(string,df,regex=False):
    return df[(df["scientificName"].str.contains(string,regex=regex,case=False)) | (df["acceptedScientificName"].str.contains(string,regex=regex,case=False))]

On définit une fonction qui renvoie le nombre de valeurs différentes dans la colonne `acceptedScientificName` d'une sous-DataFrame de `gbif`

In [878]:
def n_unique_accepted_names(df):
    return df.iloc[:,1].unique().size   

On définit une fonction qui renvoie, pour une ligne d'`iucn`, l'`acceptedScientificName` matché dans `gbif`. Elle procède de la façon suivante.
* Filtre les lignes ayant la bonne valeur pour `species`. S'il n'y a aucune ligne qui en résulte, on considère que le matching a échoué.
* Recherche les lignes matchant exactement la valeur `authority` de `iucn`. Si cela donne lieu à une unique valeur pour `acceptedScientificName`, on renvoie cette dernière, et le matching a réussi.
* Sinon, tant qu'il reste des lignes, on filtre en faisant matcher successivement les différents noms contenus dans `authority`.
* Si on arrive au fil des filtrages à une unique valeur pour `acceptedScientificName`, on renvoie cette dernière, et le matching a réussi.
* Sinon, on renvoie une des valeurs de `acceptedScientificName`, et on signale que le matching n'est pas unique ...

In [879]:
def matching_row_in_gbif(iucn_row):
    genus = iucn_row['genusName']
    species = iucn_row['speciesName']
    g_s = genus + ' ' + species
    search_res_w_g_s = gbif[(gbif['species'] == g_s)][['scientificName','acceptedScientificName']]
    if search_res_w_g_s.shape[0] == 0: # if match fails with genus+species
        print('G+S, matching failed :',g_s)
        return np.nan
    elif search_res_w_g_s.shape[0] > 1:
        authority = iucn_row.loc[["authority"]].item()
        search_res_containing_authority = search_in_gbif_names(authority,search_res_w_g_s)
        if n_unique_accepted_names(search_res_containing_authority) == 1:
            return search_res_containing_authority.iloc[0]['acceptedScientificName']
        else: # otherwise sequentially filters by words randomly drawn from authority
            words_from_authority = list(filter(lambda str: len(str) > 1,re.sub('[^\w\s]',' ',authority).split()))
            if n_unique_accepted_names(search_res_containing_authority) >= 2:
                search_res = search_res_containing_authority
            else:
                search_res = search_res_w_g_s
            while (len(words_from_authority) > 1) and (n_unique_accepted_names(search_res_w_g_s) > 1):
                random_word =  words_from_authority.pop(random.randrange(len(words_from_authority)))
                search_res_tmp = search_in_gbif_names(random_word,search_res)
                if search_res_tmp.shape[0] > 0:
                    search_res = search_res_tmp
            if search_res.shape[0] == 0:
                print('G+S+fuzzy authority, matching failed :',g_s)
                return np.nan
            else:
                if n_unique_accepted_names(search_res) > 1:
                    print('G+S+filtered authority, couln\'t find a unique match :', g_s)
                return search_res.iloc[0]['acceptedScientificName']
    else:
       return search_res_w_g_s.iloc[0,1]

In [880]:
matching_results = iucn.apply(matching_row_in_gbif,axis=1)

G+S+filtered authority, couln't find a unique match : Eriosyce crispa
G+S+filtered authority, couln't find a unique match : Cleistocactus winteri


G+S, matching failed : Rebutia albipectinata


G+S, matching failed : Melocactus braunii


G+S, matching failed : Cereus estevesii
G+S+filtered authority, couln't find a unique match : Gymnocalycium horstii


G+S+filtered authority, couln't find a unique match : Rhipsalis dissimilis


...

G+S, matching failed : Carya tomentosa
G+S, matching failed : Maytenus matudae
G+S, matching failed : Biancaea sappan


G+S, matching failed : Searsia sp. nov. A
G+S, matching failed : Conamomum pierreanum
G+S, matching failed : Conamomum rubidum
G+S, matching failed : Philenoptera kanurii


G+S, matching failed : Dupuya madagascariensis
G+S, matching failed : Dupuya haraka
G+S, matching failed : Libidibia paraguariensis
G+S, matching failed : Poissonia hypoleuca


G+S+filtered authority, couln't find a unique match : Daphne sophia
G+S+filtered authority, couln't find a unique match : Rhamnus cathartica


G+S, matching failed : Persea podadenia
G+S+filtered authority, couln't find a unique match : Persea americana


In [881]:
matching_results

0         Calocedrus rupestris Aver., T.H.Nguyên & P.K.Lôc
1        Turbinicarpus subterraneus (Backeb.) A.D.Zimme...
2                    Eriosyce esmeraldana (F.Ritter) Katt.
3               Coleocephalocereus braunii Diers & Esteves
4                 Mammillaria melaleuca Karw. ex Salm-Dyck
5                         Corryocactus tarijensis Cárdenas
6        Parodia stockingeri (Prestlé) Hofacker & P.J.B...
7                    Ferocactus flavovirens Britton & Rose
8                      Copiapoa cinerascens Britton & Rose
9                Melocactus andinus R.Gruber ex N.P.Taylor
10                Parodia nigrispina (K.Schum.) F.H.Brandt
11       Eriosyce crispa subsp. totoralensis (F.Ritter)...
12                            Matucana aureiflora F.Ritter
13                     Leptocereus arboreus Britton & Rose
14                    Cleistocactus winteri subsp. winteri
15               Echinocereus barthelowanus Britton & Rose
16                       Copiapoa serpentisulcata F.Ritt

Nombre de matchings échoués

In [882]:
np.sum(pd.isnull(matching_results))

3452

Enregistrement des matchings

In [883]:
matching_results.to_csv('matchings.csv')

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
